# Tutorial: Planning Pattern Agent Interactive Lab (Microsoft Agent Framework + Claude)

Audience:
- Data Scientists and AI Researchers who prefer notebook-first exploration.

Prerequisites:
- Basic Python and async familiarity.
- Conda env `vllm` available.
- Valid `OPENAI_API_KEY` and `SERPER_API_KEY` (default provider).

Learning goals:
- Understand each Planning component (tools, planner, executor, synthesis).
- Run an end-to-end inference from a topic to a markdown report.
- Debug common failures (missing key, invalid key, API mismatch).


## Outline

1. Environment and dependency setup
2. API key checks and sanity diagnostics
3. Interactive tool checks (`calculator`, `web_search`, `save_findings`)
4. Workflow component inspection
5. End-to-end inference run
6. Pitfalls, extensions, and exercises


In [ ]:
# Optional: run once per fresh environment
%pip install -r requirements.txt


In [ ]:
from __future__ import annotations

import os
from pathlib import Path
from datetime import datetime

from dotenv import load_dotenv

PROJECT_ROOT = Path.cwd()
if not (PROJECT_ROOT / 'main.py').exists():
    raise RuntimeError('Run this notebook from examples/planning-claude-sdk-market-research')

load_dotenv(PROJECT_ROOT / '.env')

print('Project root:', PROJECT_ROOT)
print('UTC now:', datetime.utcnow().isoformat(timespec='seconds'))


## Step 1 - Check API keys and provider alignment

This project defaults to **OpenAI** (`LLM_PROVIDER=openai`) via `OPENAI_API_KEY`.
If you switch to Anthropic, set `LLM_PROVIDER=anthropic` and use `ANTHROPIC_API_KEY`.


In [ ]:
def _key_status(name: str) -> str:
    v = os.getenv(name, '').strip()
    if not v:
        return 'MISSING'
    return 'PRESENT'

provider = os.getenv('LLM_PROVIDER', 'openai').strip().lower()
print('LLM_PROVIDER     :', provider)
print('OPENAI_API_KEY   :', _key_status('OPENAI_API_KEY'))
print('ANTHROPIC_API_KEY:', _key_status('ANTHROPIC_API_KEY'))
print('SERPER_API_KEY   :', _key_status('SERPER_API_KEY'))
print('OPENAI_MODEL     :', os.getenv('OPENAI_MODEL', 'gpt-4.1-mini'))
print('ANTHROPIC_MODEL  :', os.getenv('ANTHROPIC_MODEL', 'claude-opus-4-6'))


## Step 2 - Interact with tools directly

Before full workflow inference, test each component in isolation.


In [ ]:
import asyncio
from tools import calculator, web_search, save_findings

async def tool_smoke_test():
    print('calculator:', await calculator('((47.1 / 5.43) ** (1/6) - 1) * 100'))
    save_msg = await save_findings('tool_smoke.md', '# Tool Smoke Test\n\nSuccess')
    print('save_findings:', save_msg)

    if os.getenv('SERPER_API_KEY'):
        search_result = await web_search('AI agent market size 2026')
        print('web_search sample (first 500 chars):')
        print(search_result[:500])
    else:
        print('web_search skipped: SERPER_API_KEY missing')

await tool_smoke_test()


## Step 3 - Build and inspect planning workflow

This verifies planner -> executor -> synthesis wiring before inference.


In [ ]:
from planning_workflow import PlanningMarketResearchWorkflow

topic = 'AI agent market size 2024-2026'

if not os.getenv('LLM provider API key'):
    print('Cannot instantiate workflow: LLM provider API key missing')
    workflow = None
else:
    workflow = PlanningMarketResearchWorkflow(topic=topic)
    print('Workflow object created for topic:', topic)
    print('Internal workflow type:', type(workflow.workflow).__name__)


## Step 4 - End-to-end inference

This runs the full planning pipeline and writes a markdown report.


In [ ]:
OUTPUT_FILE = PROJECT_ROOT / 'outputs' / 'planning_notebook_report.md'

async def run_inference(topic: str):
    provider = os.getenv('LLM_PROVIDER', 'openai').strip().lower()
    if provider == 'openai' and not os.getenv('OPENAI_API_KEY'):
        raise RuntimeError('OPENAI_API_KEY is required when LLM_PROVIDER=openai')
    if provider == 'anthropic' and not os.getenv('ANTHROPIC_API_KEY'):
        raise RuntimeError('ANTHROPIC_API_KEY is required when LLM_PROVIDER=anthropic')

    wf = PlanningMarketResearchWorkflow(topic=topic)
    report = await wf.execute()

    OUTPUT_FILE.parent.mkdir(parents=True, exist_ok=True)
    OUTPUT_FILE.write_text(report, encoding='utf-8')
    return report

try:
    final_report = await run_inference(topic)
    print('Inference succeeded. Report saved to:', OUTPUT_FILE)
    print('--- Report preview (first 1000 chars) ---')
    print(final_report[:1000])
except Exception as e:
    print('Inference failed:', e)
    print('Hint: verify provider and API keys in .env')


## Common pitfalls

- **Wrong key type**: OpenAI key (`sk-proj-...`) used as `ANTHROPIC_API_KEY` causes `401 invalid x-api-key`.
- **Missing SERPER key**: planning runs but web search tool returns errors or empty data.
- **Package drift**: `agent-framework` pre-release APIs can change; pin versions when moving to production.

## Optional extensions

- Add JSON schema validation for planner output.
- Add token/cost logging per phase (planning, execution, synthesis).
- Add fallback from Planning to ReAct when steps fail repeatedly.


## Exercise

1. Change the topic to your domain (finance, healthcare, supply chain).
2. Compare output quality with and without calculator steps.
3. Record one failure mode and your mitigation.


In [ ]:
# Exercise scaffold
my_topic = 'YOUR_TOPIC_HERE'

# Uncomment to run:
# final_report = await run_inference(my_topic)
# print(final_report[:1200])
